In [1]:
#!pip install git+https://github.com/nockchun/rspy --force
import rspy as rsp
rsp.setSystemWarning(off=True)

In [2]:
import pandas as pd

# 재무제표 가져오기

In [3]:
DATA_YEAR = "2015"
DATA_QUARTER = "4Q"
EXCEL_SHEET = 1

In [4]:
df_origin = pd.read_excel(f"{DATA_YEAR}_{DATA_QUARTER}_재무제표.xlsx", sheet_name=EXCEL_SHEET)

In [ ]:
df_origin.head()

# 데이터 전처리

In [6]:
df = df_origin.fillna(0)
df["보고년도"] = DATA_YEAR
df["연결유무"] = False

In [7]:
df = df.rename(columns={
    "당기 1분기말" : "당기",
    "당기 2분기말" : "당기",
    "당기 반기말" : "당기",
    "당기 3분기말" : "당기",
    "보고서종류" : "보고분기",
    "전기말" : "전기",
    "전전기말" : "전전기"
})

In [ ]:
df.head(5)

In [9]:
def fixData(row):
    row.연결유무 = row.재무제표종류.split("-")[-1] == "연결재무제표"
    row.종목코드 = row.종목코드[1:-1]
    row.보고분기 = DATA_QUARTER
    row.항목명 = row.항목명.strip()
    return row;

In [10]:
df = df.apply(fixData, axis=1)

In [11]:
df = df[[
    "종목코드", "연결유무", "보고년도", "보고분기", "항목명", "당기"
]]
df = df.astype({
    "종목코드":"str", "연결유무":"boolean", "보고년도":"str", "보고분기":"str", "항목명":"str", "당기":"int64"
})

In [12]:
df.head()

,종목코드,연결유무,보고년도,보고분기,항목명,당기
0,000040,True,2015,4Q,자산 [abstract],0
1,000040,True,2015,4Q,유동자산,81382258480
2,000040,True,2015,4Q,현금및현금성자산,11774961508
3,000040,True,2015,4Q,단기금융상품,0
4,000040,True,2015,4Q,매출채권및기타채권,37920290110


# DB에 저장.

In [13]:
rows = []
for index, row in df.iterrows():
    if row.당기 == 0:
        continue
    rows.append(tuple(row))

In [14]:
rows[:5]

[('000040', True, '2015', '4Q', '유동자산', 81382258480),
 ('000040', True, '2015', '4Q', '현금및현금성자산', 11774961508),
 ('000040', True, '2015', '4Q', '매출채권및기타채권', 37920290110),
 ('000040', True, '2015', '4Q', '재고자산', 31002757518),
 ('000040', True, '2015', '4Q', '기타유동자산', 684249344)]

In [15]:
db = rsp.PGManager()

In [16]:
db.createPool("postgres", "postgres", "postgres", "192.168.0.200")
db.setMapper("mapper.xml")

In [17]:
db.executeMany("inStateBalance", rows)